In [26]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import to_categorical # FIX ask if this is allowed
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier # for hyperparam tuning only
from tensorflow.keras.optimizers import SGD
import os
from tensorflow import keras
import time


In [14]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
assert X_train.shape == (50000, 32, 32, 3)
assert X_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)

In [15]:
X_train, X_test = X_train/255, X_test/255

In [16]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42, stratify=y_train)
print("Training set shapes:")
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("\nValidation set shapes:")
print("X_val:", X_val.shape)
print("y_val:", y_val.shape)
print("\nTest set shapes:")
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

Training set shapes:
X_train: (42500, 32, 32, 3)
y_train: (42500, 1)

Validation set shapes:
X_val: (7500, 32, 32, 3)
y_val: (7500, 1)

Test set shapes:
X_test: (10000, 32, 32, 3)
y_test: (10000, 1)


In [17]:
def create_mlp(kernel_init = 'he_uniform', drop_out_value = 0.2, learning_rate_schedule = 'exponential'):
    '''A function to create the mlp model'''
    model_mlp = keras.models.Sequential([
        keras.layers.Flatten(input_shape = [32, 32, 3]),
        keras.layers.Dropout(rate = drop_out_value),
        keras.layers.Dense(1500, activation = "relu", kernel_initializer = kernel_init),
        keras.layers.Dropout(rate = drop_out_value),
        keras.layers.Dense(1000, activation = "relu", kernel_initializer = kernel_init),
        keras.layers.Dropout(rate = drop_out_value),
        keras.layers.Dense(500, activation = "relu", kernel_initializer = kernel_init),
        keras.layers.Dropout(rate = drop_out_value),
        keras.layers.Dense(10, activation = "softmax")
    ])

    if learning_rate_schedule == 'power':
        optimizer = keras.optimizers.SGD(learning_rate = 0.01, decay = 1e-4)
    elif learning_rate_schedule == 'exponential':
        s = 100 * len(X_train) // 32 # number of steps in 20 epochs (batch size = 32)
        learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
        optimizer = keras.optimizers.SGD(learning_rate)
    model_mlp.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model_mlp

In [19]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)


In [27]:
if os.path.exists('Zhang_Hanlin-MLP'):
    #model_mlp = keras.models.model_from_json(json.dumps('Zhang_Hanlin-MLP'))
    model_mlp = tf.keras.models.load_model('Zhang_Hanlin-MLP')
    model_mlp.summary()
    #model_mlp.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
    history_mlp = model_mlp.fit(X_train, y_train, epochs = 1, validation_data = (X_val, y_val), callbacks = [early_stopping])
else:
    print("Can not find the MLP model in current dir!")
    model_mlp = create_mlp(kernel_init = 'he_uniform', drop_out_value = 0.2, learning_rate_schedule = 'exponential')
    model_mlp.summary()
    mlp_start = time.time()
    history_mlp = model_mlp.fit(X_train, y_train, epochs = 100, validation_data = (X_val, y_val), callbacks = [early_stopping])
    mlp_end = time.time()
    model_mlp.save('Zhang_Hanlin-MLP')

Can not find the MLP model in current dir!
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dropout_4 (Dropout)         (None, 3072)              0         
                                                                 
 dense_4 (Dense)             (None, 1500)              4609500   
                                                                 
 dropout_5 (Dropout)         (None, 1500)              0         
                                                                 
 dense_5 (Dense)             (None, 1000)              1501000   
                                                                 
 dropout_6 (Dropout)         (None, 1000)              0         
                                                                 
 dense_6 (D

2023-09-03 21:29:43.069433: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1329/1329 [==============================] - ETA: 0s - loss: 3.2009 - accuracy: 0.2109

2023-09-03 21:30:05.574547: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1329/1329 [==============================] - 24s 16ms/step - loss: 3.2009 - accuracy: 0.2109 - val_loss: 2.7455 - val_accuracy: 0.2255
Epoch 2/100
1329/1329 [==============================] - 17s 13ms/step - loss: 2.4542 - accuracy: 0.2525 - val_loss: 3.0099 - val_accuracy: 0.2149
Epoch 3/100
1329/1329 [==============================] - 17s 12ms/step - loss: 2.4187 - accuracy: 0.2557 - val_loss: 6.3834 - val_accuracy: 0.1552
Epoch 4/100
1329/1329 [==============================] - 17s 13ms/step - loss: 2.8244 - accuracy: 0.2430 - val_loss: 7.3319 - val_accuracy: 0.1311
Epoch 5/100
1329/1329 [==============================] - 16s 12ms/step - loss: 3.7812 - accuracy: 0.2240 - val_loss: 8.2117 - val_accuracy: 0.1121
Epoch 6/100
1329/1329 [==============================] - 19s 14ms/step - loss: 6.2866 - accuracy: 0.2076 - val_loss: 28.4749 - val_accuracy: 0.1059
Epoch 7/100
1329/1329 [==============================] - 17s 13ms/step - loss: 11.0960 - accuracy: 0.2033 - val_loss: 41.0877 - v